In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
from torchsummary import summary
import copy

import sys
sys.path.append("../src/")
import model

In [3]:
vgg = torchvision.models.vgg19()

In [4]:
summary(vgg, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [17]:
def replace_layer(model, keys=None):
    model = copy.deepcopy(model)
    for name, layer in model.named_modules():
        # Replace maxpool with avgpool
        if "AVG" in keys and isinstance(layer, nn.MaxPool2d):
            print("Replace {} with {}".format(name, "avgPool"))
            first_wrap_name, second_wrap_name = name.split(".")
            second_wrap_name = int(second_wrap_name)
            kernel_size = layer.kernel_size
            stride = layer.stride
            padding = layer.padding
            ceil_mode = layer.ceil_mode
            model._modules[first_wrap_name][second_wrap_name] = nn.AvgPool2d(kernel_size=kernel_size, stride=stride, 
                                                                           padding=padding, ceil_mode=ceil_mode)
        if "LEAK" in keys and isinstance(layer, nn.ReLU):
            print("Replace {} with {}".format(name, "LeakReLU"))
            first_wrap_name, second_wrap_name = name.split(".")
            second_wrap_name = int(second_wrap_name)
            model._modules[first_wrap_name][second_wrap_name] = nn.LeakyReLU() 
    return model

In [18]:
keys = ["AVG"]
vgg_avg = replace_layer(vgg, keys)
summary(vgg_avg, (3, 224, 224))

Replace features.4 with avgPool
Replace features.9 with avgPool
Replace features.18 with avgPool
Replace features.27 with avgPool
Replace features.36 with avgPool
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         AvgPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        AvgPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
    

In [14]:
keys = ["LEAK"]
vgg_leak = replace_layer(vgg, keys)
summary(vgg_leak, (3, 224, 224))

Replace features.1 with LeakReLU
Replace features.3 with LeakReLU
Replace features.6 with LeakReLU
Replace features.8 with LeakReLU
Replace features.11 with LeakReLU
Replace features.13 with LeakReLU
Replace features.15 with LeakReLU
Replace features.17 with LeakReLU
Replace features.20 with LeakReLU
Replace features.22 with LeakReLU
Replace features.24 with LeakReLU
Replace features.26 with LeakReLU
Replace features.29 with LeakReLU
Replace features.31 with LeakReLU
Replace features.33 with LeakReLU
Replace features.35 with LeakReLU
Replace classifier.1 with LeakReLU
Replace classifier.4 with LeakReLU
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
         LeakyReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
         LeakyReLU-4         [-1, 64, 224, 224]               0
     

In [15]:
keys = ["LEAK", "AVG"]
vgg_leak = replace_layer(vgg, keys)
summary(vgg_leak, (3, 224, 224))

Replace features.1 with LeakReLU
Replace features.3 with LeakReLU
features.4 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Replace features.6 with LeakReLU
Replace features.8 with LeakReLU
features.9 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Replace features.11 with LeakReLU
Replace features.13 with LeakReLU
Replace features.15 with LeakReLU
Replace features.17 with LeakReLU
features.18 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Replace features.20 with LeakReLU
Replace features.22 with LeakReLU
Replace features.24 with LeakReLU
Replace features.26 with LeakReLU
features.27 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Replace features.29 with LeakReLU
Replace features.31 with LeakReLU
Replace features.33 with LeakReLU
Replace features.35 with LeakReLU
features.36 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Replace classifier.1 with L

In [7]:
var = torch.tensor([1,2,3,4,5,6,100.0])

In [8]:
torch.max(var)

tensor(100.)

In [9]:
torch.min(var)

tensor(1.)

In [10]:
torch.norm(var, p=1)

tensor(121.)

In [11]:
torch.norm(var, p=2)

tensor(100.4540)

In [14]:
torch.sqrt(torch.sum(var ** 2))

tensor(100.4540)